<span style="color: rgb(99, 100, 102); font-family: Roboto, sans-serif; background-color: rgb(255, 255, 255);">A burglar sees 8 items, of different worths and weights. He wants to take the items of greatest total value whose total weight is not more than the maximum&nbsp;</span>  _WTMAX_  <span style="color: rgb(99, 100, 102); font-family: Roboto, sans-serif; background-color: rgb(255, 255, 255);">&nbsp;he can carry.</span>

[Source](https:\www.fico.com\fico-xpress-optimization\docs\dms2020-01\mosel\UG\dhtml\moselugA2_sec_secA2burglar.html)<span style="background-color: rgb(255, 255, 255);"><br></span>

See the first 1000 rows

In [3]:
SELECT TOP (1000) [Id]
      ,[value]
      ,[weight]
  FROM [portfolio].[burglars].[items]

(8 rows affected)

Total execution time: 00:00:00.005

Id,value,weight
1,15,2
2,100,20
3,90,20
4,60,30
5,40,40
6,15,30
7,10,60
8,1,10


Select the conditions

In [4]:
  SELECT TOP (1000) [Id]
      ,[wtmax ]
      ,[itemsnumber]
  FROM [portfolio].[burglars].[conditions]

(1 row affected)

Total execution time: 00:00:00.004

Id,wtmax,itemsnumber
1,102,8


Select all the rows having weight less than the maximum weight

In [7]:
SELECT TOP (1000) [Id]
      ,[value]
      ,[weight]
  FROM [portfolio].[burglars].[items]
  WHERE [weight] <  (SELECT TOP (1000) [wtmax ] FROM [portfolio].[burglars].[conditions])
  ORDER BY [weight] DESC

(8 rows affected)

Total execution time: 00:00:00.020

Id,value,weight
7,10,60
5,40,40
6,15,30
4,60,30
2,100,20
3,90,20
8,1,10
1,15,2


In [9]:
SELECT [Id]
      ,[value]
      ,[weight]
FROM [portfolio].[burglars].[items] i
WHERE (SELECT SUM([weight]) FROM [portfolio].[burglars].[items] WHERE [Id] <= i.[Id]) <= (SELECT TOP (1000) [wtmax ] FROM [portfolio].[burglars].[conditions]);

(4 rows affected)

Total execution time: 00:00:00.024

Id,value,weight
1,15,2
2,100,20
3,90,20
4,60,30


In [5]:
WITH RecursiveCTE AS (
    SELECT
        CAST([Id] AS VARCHAR(MAX)) AS Combination,
        [value] AS TotalValue,
        [weight] AS TotalWeight
    FROM
        [portfolio].[burglars].[items]
    UNION ALL
    SELECT
        rc.Combination + ',' + CAST(i.Id AS VARCHAR(MAX)),
        rc.TotalValue + i.[value],
        rc.TotalWeight + i.[weight]
    FROM
        RecursiveCTE rc
    JOIN
        [portfolio].[burglars].[items] i ON i.Id > CAST(SUBSTRING(rc.Combination, LEN(rc.Combination) - CHARINDEX(',', REVERSE(rc.Combination)) + 2, LEN(rc.Combination)) AS INT)
    WHERE
        rc.TotalWeight + i.[weight] <= (SELECT [wtmax] FROM [portfolio].[burglars].[conditions])
)
SELECT
    Combination,
    TotalValue,
    TotalWeight
FROM
    RecursiveCTE
WHERE
    TotalWeight <= (SELECT [wtmax ] FROM [portfolio].[burglars].[conditions])
ORDER BY
    TotalValue DESC, TotalWeight;

(610 rows affected)

Total execution time: 00:00:00.104

Combination,TotalValue,TotalWeight
"2,1,2,3,4,8",366,102
"2,1,2,3,4",365,92
"3,1,2,3,4,8",356,102
"3,1,2,3,4",355,92
"2,2,3,4,8",351,100
"2,2,3,4",350,90
"2,1,2,3,5",345,102
"3,2,3,4,8",341,100
"3,2,3,4",340,90
"3,1,2,3,5",335,102


In the results there are repetitions in the combinations.

In [6]:
WITH RecursiveCTE AS (
    SELECT
        CAST([Id] AS VARCHAR(MAX)) AS Combination,
        [value] AS TotalValue,
        [weight] AS TotalWeight
    FROM
        [portfolio].[burglars].[items]
    UNION ALL
    SELECT
        rc.Combination + ',' + CAST(i.Id AS VARCHAR(MAX)),
        rc.TotalValue + i.[value],
        rc.TotalWeight + i.[weight]
    FROM
        RecursiveCTE rc
    JOIN
        [portfolio].[burglars].[items] i ON i.Id > CAST(SUBSTRING(rc.Combination, LEN(rc.Combination) - CHARINDEX(',', REVERSE(rc.Combination)) + 2, LEN(rc.Combination)) AS INT)
    WHERE
        rc.TotalWeight + i.[weight] <= (SELECT [wtmax] FROM [portfolio].[burglars].[conditions])
    AND NOT EXISTS (
            SELECT 1
            FROM [portfolio].[burglars].[items] ni
            WHERE ni.Id = i.Id
            AND ',' + rc.Combination + ',' LIKE '%,' + CAST(ni.Id AS VARCHAR(MAX)) + ',%'
        )
)
SELECT
    Combination,
    TotalValue,
    TotalWeight
FROM
    RecursiveCTE
WHERE
    TotalWeight <= (SELECT [wtmax ] FROM [portfolio].[burglars].[conditions])
ORDER BY
    TotalValue DESC, TotalWeight;

(392 rows affected)

Total execution time: 00:00:00.290

Combination,TotalValue,TotalWeight
"6,1,2,3,4",280,102
"4,1,2,3,6",280,102
"3,1,2,4,6",280,102
"2,1,3,4,6",280,102
"1,2,3,4,6",280,102
"1,2,3,4,8",266,82
"2,1,3,4,8",266,82
"3,1,2,4,8",266,82
"4,1,2,3,8",266,82
"8,1,2,3,4",266,82
